In [1]:
WORKOUT_SUGGEST_CHAIN = \
{"inputs":["chat_history", "time_unit", "timely", "day_number", 'medical_record', 'health_goal', 'timeline', 'input'],
 "outputs": {"exercises": "a js array of excercises for the workout routine.",
             "frequency": "a js array of each excercise frequency.",
             "description": "a description including the health usage of each excercise."},
 "template": """You are a professional personal trainner.
 
 You have {timely} checkup with your customer and today is day number {day_number}.
 {chat_history}
 
 Your customer response to you: "{input}".
 
 You have noted your customer's history medical record and health goals.
Medical record: {medical_record}.
Health goals: {health_goal}.

Mission: Based on what your customer have told and what you have advised so far, please plan the next {time_unit} workout routine for your customer.
Requirements: You must adhere to the requirements listed in triple backquotes.
```
The workout routine should contain easy to do exercises that can be done at home.
You should note in mind your customer's health goals, in order to devise a suitable workout plan.
You should check carefully your customer's medical record before making any decision. Avoid including any exercises that your customer should not do.
Your workout routine should be fully thought and well planned so that your customer can achieve the health goal in {timeline} provided that your customer do the routine {timely}. 
```

{format_instructions}

Output:"""}

from langchain.llms import BaseLLM
from langchain import LLMChain, PromptTemplate
import sys
import os
from typing import Dict
# sys.path.append(f"{os.path.dirname(__file__)}/../..")
sys.path.append('../')
from botcore.utils import build_prompt, CheckupMemory, parse_nested_json
from botcore.setup import trace_palm2

def build_workout_suggest_chain(model, memory):
    inputs = WORKOUT_SUGGEST_CHAIN['inputs']
    outputs = WORKOUT_SUGGEST_CHAIN['outputs']
    template = WORKOUT_SUGGEST_CHAIN['template']
    prompt = build_prompt(inputs, outputs, template)
    chain = LLMChain(llm=model, prompt=prompt, output_key='result', verbose=False, memory=memory)
    return chain

In [2]:
from botcore.setup import trace_palm2
model = trace_palm2(max_tokens=200)
memory = CheckupMemory(input_key='input', role='professional personal trainner')

chain = build_workout_suggest_chain(model, memory.get())

Enable tracing at test-deploy
Vertex AI Palm 2 ready


In [7]:
from botcore.utils import load_example_input
input = load_example_input()

ans = chain({"time_unit": "week", "timely": "weekly", "day_number": 1, "input": "Hello!", **input})
ans



> Entering new  chain...
Prompt after formatting:
You are a professional personal trainner.
 
 You have weekly checkup with your customer and today is day number 1.
 
 Your customer response to you: "Hello!".
 
 You have noted your customer's history medical record and health goals.
Medical record: No chronic diseases or food allergies..
Health goals: Weight loss and improving overall cardiovascular health..

Mission: Based on what your customer have told and what you have advised so far, please plan the next week workout routine for your customer.
Requirements: You must adhere to the requirements listed in triple backquotes.
```
The workout routine should contain easy to do exercises that can be done at home.
You should note in mind your customer's health goals, in order to devise a suitable workout plan.
You should check carefully your customer's medical record before making any decision. Avoid including any exercises that your customer should not do.
Your workout routine should be

{'time_unit': 'week',
 'timely': 'weekly',
 'day_number': 1,
 'input': 'Hello!',
 'medical_record': 'No chronic diseases or food allergies.',
 'eating_habit': 'Likes fast food and quick-prep meals but wants to switch to a healthier eating regimen',
 'health_goal': 'Weight loss and improving overall cardiovascular health.',
 'timeline': '2 to 3 months.',
 'result': '```json\n{\n  "exercises": ["Walking", "Squats", "Push-ups", "Plank"],\n  "frequency": ["3 times a week", "3 times a week", "3 times a week", "3 times a week"],\n  "description": "Walking is a great way to get in shape and improve your cardiovascular health. It\'s also a low-impact exercise, so it\'s easy on your joints. Squats are a great way to strengthen your legs and glutes. Push-ups are a great way to build upper body strength. Planks are a great way to work your core and improve your balance."\n}\n```'}

In [8]:
from botcore.utils import parse_nested_json
data = parse_nested_json(ans['result'])
data

{'exercises': ['Walking', 'Squats', 'Push-ups', 'Plank'],
 'frequency': ['3 times a week',
  '3 times a week',
  '3 times a week',
  '3 times a week'],
 'description': "Walking is a great way to get in shape and improve your cardiovascular health. It's also a low-impact exercise, so it's easy on your joints. Squats are a great way to strengthen your legs and glutes. Push-ups are a great way to build upper body strength. Planks are a great way to work your core and improve your balance."}